In [120]:
from numpy import save, load
from pandas import read_csv
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [121]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/'  #change dir to your project folder

import numpy as np
x_train = np.load('gdrive/My Drive/dataset/dataset5/trainX.npy')
y_train = np.load('gdrive/My Drive/dataset/dataset5/trainy.npy')
x_test = np.load('gdrive/My Drive/dataset/dataset5/testX.npy')
y_test = np.load('gdrive/My Drive/dataset/dataset5/testy.npy')
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)
'''
x_train = np.load('gdrive/My Drive/dataset1/acc+gyr/trainX.npy')
y_train = np.load('gdrive/My Drive/dataset1/acc+gyr/trainy.npy')
x_test = np.load('gdrive/My Drive/dataset1/acc+gyr/testX.npy')
y_test = np.load('gdrive/My Drive/dataset1/acc+gyr/testy.npy')
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)
'''

Mounted at /content/drive
Mounted at /content/gdrive
(66542, 256, 6) (66542, 2) (7600, 256, 6) (7600, 2)


"\nx_train = np.load('gdrive/My Drive/dataset1/acc+gyr/trainX.npy')\ny_train = np.load('gdrive/My Drive/dataset1/acc+gyr/trainy.npy')\nx_test = np.load('gdrive/My Drive/dataset1/acc+gyr/testX.npy')\ny_test = np.load('gdrive/My Drive/dataset1/acc+gyr/testy.npy')\nprint(x_train.shape, y_train.shape, x_test.shape, y_test.shape)\n"

In [122]:
idx = np.random.permutation(len(x_train))
x_train = x_train[idx]
y_train = y_train[idx]

In [123]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Attention and Normalization
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(res)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    return x + res

In [124]:
'''
def transformer_decoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    #x = layers.Embedding(input_dim=inputs.shape[:] , output_dim=inputs.shape[:])
    x = layers.Embedding(1000, 64, input_length=10)(inputs, inputs)

    # Attention and Normalization
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(res)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1,activation="relu")(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    return x + res
'''

In [125]:
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(2, activation="softmax")(x)
    return keras.Model(inputs, outputs)

In [126]:
input_shape = x_train.shape[1:]

model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)

model.compile(
    loss="categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=["accuracy"],
)
model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_21 (InputLayer)          [(None, 256, 6)]     0           []                               
                                                                                                  
 multi_head_attention_33 (Multi  (None, 256, 6)      27654       ['input_21[0][0]',               
 HeadAttention)                                                   'input_21[0][0]']               
                                                                                                  
 dropout_70 (Dropout)           (None, 256, 6)       0           ['multi_head_attention_33[0][0]']
                                                                                                  
 layer_normalization_64 (LayerN  (None, 256, 6)      12          ['dropout_70[0][0]']       

In [127]:
history = model.fit(
    x_train,
    y_train,
    verbose=True,
    validation_split=0.2,
    epochs=10,
    batch_size=128,
    callbacks=callbacks,
)

Epoch 1/10
416/416 [==============================] - 487s 1s/step - loss: 0.5779 - accuracy: 0.6655 - val_loss: 0.5715 - val_accuracy: 0.6947
Epoch 2/10
416/416 [==============================] - 481s 1s/step - loss: 0.5101 - accuracy: 0.7197 - val_loss: 0.5316 - val_accuracy: 0.7299
Epoch 3/10
416/416 [==============================] - 481s 1s/step - loss: 0.4904 - accuracy: 0.7323 - val_loss: 0.5508 - val_accuracy: 0.7211
Epoch 4/10
416/416 [==============================] - 482s 1s/step - loss: 0.4844 - accuracy: 0.7342 - val_loss: 0.5876 - val_accuracy: 0.7234
Epoch 5/10
416/416 [==============================] - 482s 1s/step - loss: 0.4820 - accuracy: 0.7353 - val_loss: 0.5667 - val_accuracy: 0.7291
Epoch 6/10
416/416 [==============================] - 481s 1s/step - loss: 0.4764 - accuracy: 0.7373 - val_loss: 0.5235 - val_accuracy: 0.7173
Epoch 7/10
416/416 [==============================] - 482s 1s/step - loss: 0.4718 - accuracy: 0.7435 - val_loss: 0.5728 - val_accuracy: 0.7098

In [128]:
loss, accuracy = model.evaluate(x_test, y_test, verbose=False)
print("Testing Accuracy: {:.4f}".format(accuracy))


Testing Accuracy: 0.7138
